In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import KNNImputer
import pickle

# Data

In [21]:
df = pd.read_pickle("./UV_all(0720).pkl")
df.head(2)

,,lon,lat,uv,band1,band2,band3,band4,band5,band6,band7,...,band12,band13,band14,band15,band16,solarza,sateza,esr,height,landtype
stn,date_time,,,,,,,,,,,,,,,,,,,,,
13,2020-01-01,126.16000,33.30000,0.0,0.00000,0.00000,0.0000,-0.00075,0.0000,-0.00065,268.21695,...,250.75924,269.97608,269.70811,268.61702,259.10334,165.17963,38.96359,-9.5091,71.00,0
105,2020-01-01,128.89098,37.75147,0.0,-0.00057,0.00057,0.0003,-0.00030,0.0003,0.00000,268.21695,...,246.67115,268.22810,268.41686,268.01723,256.99670,163.20549,43.95463,-9.4093,26.04,3


# Missing data

In [22]:
df.replace(-999.0, np.nan, inplace=True)
df.reset_index(inplace=True)
df.head()

,stn,date_time,lon,lat,uv,band1,band2,band3,band4,band5,...,band12,band13,band14,band15,band16,solarza,sateza,esr,height,landtype
0,13,2020-01-01,126.16000,33.30000,0.0,0.00000,0.00000,0.00000,-0.00075,0.0000,...,250.75924,269.97608,269.70811,268.61702,259.10334,165.17963,38.96359,-9.50910,71.00,0
1,105,2020-01-01,128.89098,37.75147,0.0,-0.00057,0.00057,0.00030,-0.00030,0.0003,...,246.67115,268.22810,268.41686,268.01723,256.99670,163.20549,43.95463,-9.40930,26.04,3
2,108,2020-01-01,126.96500,37.57100,NaN,0.00000,0.00000,-0.00089,-0.00075,-0.0006,...,241.24994,258.33982,258.18692,257.58363,250.50021,162.48279,43.77999,-9.37761,85.50,3
3,112,2020-01-01,126.62490,37.47772,0.0,-0.00057,-0.00057,-0.00060,0.00015,0.0000,...,241.55488,258.54927,258.64012,258.03883,251.00820,162.39995,43.67046,-9.37422,68.99,3
4,115,2020-01-01,130.89864,37.48129,0.0,0.00000,-0.00057,-0.00030,0.00075,0.0006,...,244.56708,264.88004,263.95472,262.33534,253.25397,164.19725,43.74521,-9.45200,222.80,4


In [23]:
df.isnull().sum()

stn               0
date_time         0
lon               0
lat               0
uv           104478
band1         18435
band2         18435
band3         18435
band4         18435
band5         18435
band6         18435
band7         18441
band8         18435
band9         18435
band10        18435
band11        18435
band12        18435
band13        18435
band14        18435
band15        18435
band16        18435
solarza       51855
sateza            0
esr           51855
height            0
landtype          0
dtype: int64

# Train, Test split

In [30]:
df_train = df.loc[(df["date_time"] >= '2020-01-01') & (df["date_time"] < '2022-01-01')]
df_test = df.loc[(df["date_time"] >= '2022-06-01') & (df["date_time"] < '2022-07-01')]

In [31]:
df_test = df_test.drop(columns=['uv', 'solarza', 'esr'])

In [32]:
df_test

,stn,date_time,lon,lat,band1,band2,band3,band4,band5,band6,...,band10,band11,band12,band13,band14,band15,band16,sateza,height,landtype
1578960,13,2022-06-01,126.16000,33.30000,0.00000,0.00000,-0.00060,0.00060,-0.0003,-0.00065,...,240.31673,251.77517,241.07130,251.49092,250.11913,248.78686,243.12601,38.96359,71.00,0
1578961,105,2022-06-01,128.89098,37.75147,-0.00057,-0.00057,0.00000,-0.00030,0.0000,0.00000,...,239.77969,262.07101,244.35010,262.71227,261.14580,257.52658,246.95577,43.95463,26.04,3
1578962,108,2022-06-01,126.96500,37.57100,0.00000,-0.00057,-0.00030,0.00045,0.0000,0.00000,...,246.10603,271.27599,249.24111,271.81756,270.25616,266.99858,254.94651,43.77999,85.50,3
1578963,112,2022-06-01,126.62490,37.47772,0.00000,0.00000,0.00060,0.00045,-0.0003,0.00000,...,247.21581,272.76683,249.94977,273.52547,272.33661,269.12853,256.15036,43.67046,68.99,3
1578964,115,2022-06-01,130.89864,37.48129,0.00000,0.00000,0.00089,-0.00030,0.0003,0.00000,...,236.01801,253.63890,239.88321,253.29031,251.46606,249.01625,240.92761,43.74521,222.80,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630810,146,2022-06-25,127.11900,35.84080,0.00000,-0.00057,-0.00089,0.00015,0.0000,0.00000,...,261.81858,284.18772,258.49191,286.87485,287.19605,285.48855,271.35400,41.81018,62.90,3
1630811,152,2022-06-25,129.33472,35.58250,-0.00057,0.00000,-0.00030,-0.00030,-0.0003,0.00000,...,264.16538,290.82040,263.99872,293.32422,293.51096,291.15861,274.60342,41.52495,82.00,2
1630812,156,2022-06-25,126.89158,35.17294,0.00000,0.00000,0.00089,-0.00030,-0.0003,-0.00065,...,263.87579,287.48170,261.53112,290.14436,290.23278,288.05655,273.15633,41.04849,72.38,3
1630813,159,2022-06-25,129.03203,35.10468,0.00000,0.00057,0.00030,-0.00075,-0.0003,0.00000,...,258.90027,285.46511,260.95627,287.80066,287.63090,285.01340,270.36623,40.96396,69.56,3


# 지점별 KNN imputer

### Train

In [34]:
df_imputed = pd.DataFrame(columns=df_train.columns)
stn_list = list(df_train["stn"].unique())

for stn in tqdm(stn_list):
    df_train_1 = df_train.loc[df_train["stn"] == stn]
    df_train_1.set_index("date_time", inplace=True)
    imputer = KNNImputer(n_neighbors=4)
    np_imputed = imputer.fit_transform(df_train_1)
    df_train_2 = pd.DataFrame(np_imputed, columns=df_train_1.columns)
    df_train_1.reset_index(inplace=True)
    df_train_2["date_time"] = df_train_1["date_time"]
    df_imputed = pd.concat([df_imputed, df_train_2], axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [14:33<00:00, 58.22s/it]


# Test

In [17]:
df_test_imputed = pd.DataFrame(columns=df_test.columns)
stn_list = list(df_test["stn"].unique())

for stn in tqdm(stn_list):
    df_test_1 = df_test.loc[df_test["stn"] == stn]
    df_test_1.set_index("date_time", inplace=True)
    imputer = KNNImputer(n_neighbors=4)
    np_imputed = imputer.fit_transform(df_test_1)
    df_test_2 = pd.DataFrame(np_imputed, columns=df_test_1.columns)
    df_test_1.reset_index(inplace=True)
    df_test_2["date_time"] = df_test_1["date_time"]
    df_test_imputed = pd.concat([df_test_imputed, df_test_2], axis=0)


  0%|                                                                                           | 0/15 [00:00<?, ?it/s]


ValueError: Shape of passed values is (3457, 22), indices imply (3457, 24)

In [ ]:
stn_list

# Train, Test 합치기

In [ ]:
all_impute = pd.concat([df_imputed, df_test_imputed], axis=0)

In [ ]:
all_impute.info()

In [ ]:
all_impute = all_impute.astype({'stn':'int64', 'date_time':'datetime64[ns]', 'lon':'float64', 'lat':'float64',
                                'uv':'float64', 'band1':'float64', 'band2':'float64', 'band3':'float64', 
                                'band4':'float64', 'band5':'float64', 'band6':'float64', 'band7':'float64',
                                'band8':'float64', 'band9':'float64', 'band10':'float64', 'band11':'float64',
                                'band12':'float64', 'band13':'float64', 'band14':'float64', 'band15':'float64',
                                'band16':'float64', 'solarza':'float64', 'sateza':'float64', 'esr':'float64',
                                'height':'float64', 'landtype':'int64'})

In [ ]:
with open("knn_imp(0720).pkl", "wb") as f:
    pickle.dump(all_impute, f)

In [2]:
print( pd.__version__ )

1.4.2


In [3]:
df = pd.read_pickle('knn_imp(0720).pkl')

In [6]:
df["date_time"]

0      2020-01-01 00:00:00
1      2020-01-01 00:10:00
2      2020-01-01 00:20:00
3      2020-01-01 00:30:00
4      2020-01-01 00:40:00
               ...        
3452   2022-06-24 23:20:00
3453   2022-06-24 23:30:00
3454   2022-06-24 23:40:00
3455   2022-06-24 23:50:00
3456   2022-06-25 00:00:00
Name: date_time, Length: 1630815, dtype: datetime64[ns]

In [5]:
df.to_csv("knn_imp(0720).csv")